In [2]:
import os
import glob
import numpy as np
import cv2
from PIL import Image
import math

import PIL
from rembg import remove
from preprocessing.image_processing import remove_background_from_folder, remove_background_recursive

## Background Removal
- Remove the background of the image and save image as RGBA without a background

In [3]:
target_folder = "example_data/Meshfleet_Eval/Results_002"
remove_background_recursive(target_folder)

Processing background removal in folder: example_data/Meshfleet_Eval/Results_002/043c94a4e37fc017a9c602bb6a28e8f404b1e99e8c4f3c9a6d4e0e997a8f9359
Background removal complete in folder: example_data/Meshfleet_Eval/Results_002/043c94a4e37fc017a9c602bb6a28e8f404b1e99e8c4f3c9a6d4e0e997a8f9359
Processing background removal in folder: example_data/Meshfleet_Eval/Results_002/011880bbcfe10922fac6888eddacacfa1daec200178c977b3d6beb0e6c8802c6
Background removal complete in folder: example_data/Meshfleet_Eval/Results_002/011880bbcfe10922fac6888eddacacfa1daec200178c977b3d6beb0e6c8802c6
Processing background removal in folder: example_data/Meshfleet_Eval/Results_002/082d8c03d113d9af118493039bf727858382eb5f01e5a2cc5fa5df5327d44c6d
Background removal complete in folder: example_data/Meshfleet_Eval/Results_002/082d8c03d113d9af118493039bf727858382eb5f01e5a2cc5fa5df5327d44c6d
Processing background removal in folder: example_data/Meshfleet_Eval/Results_002/07ed30b7de006ad2e2bd14bee35cc1fcde860b36326e76ccd

## Front View Detection

In [4]:
import torch
from preprocessing.image_processing import align_views, load_dino, load_dino_model, get_dino_transform, release_gpu
# from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"

"""
vlm_model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b")
vlm_processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")
vlm_model.to(device)
"""

dino_model = load_dino(device)
dino_transform = get_dino_transform(224)

Using cache found in /home/q613871/.cache/torch/hub/facebookresearch_dinov2_main
/home/q613871/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/q613871/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/q613871/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [6]:
gt_folder = "example_data/Meshfleet_Eval/Ground_Truth"
generated_folder = "example_data/Meshfleet_Eval/Results_002"
prompt = "Is this a front view or a rear view?"
df = align_views(generated_folder, detection_mode="dino", dino_model=dino_model, dino_transform=dino_transform, ground_truth_parent=gt_folder, device=device)

DINO similarity: Candidate1: 0.9210, Candidate2: 0.6608
[043c94a4e37fc017a9c602bb6a28e8f404b1e99e8c4f3c9a6d4e0e997a8f9359] Detected front view index: 3 (original file: 003.png)
DINO similarity: Candidate1: 0.9320, Candidate2: 0.7395
[011880bbcfe10922fac6888eddacacfa1daec200178c977b3d6beb0e6c8802c6] Detected front view index: 3 (original file: 003.png)
DINO similarity: Candidate1: 0.8266, Candidate2: 0.6139
[082d8c03d113d9af118493039bf727858382eb5f01e5a2cc5fa5df5327d44c6d] Detected front view index: 3 (original file: 003.png)
DINO similarity: Candidate1: 0.8571, Candidate2: 0.5657
[07ed30b7de006ad2e2bd14bee35cc1fcde860b36326e76ccdbec8eb61b63aac8] Detected front view index: 3 (original file: 003.png)
DINO similarity: Candidate1: 0.6760, Candidate2: 0.9394
[0082c4683a107e2859d4ad4f1889080acb1df32c2e0507c048f3eacf06c97300] Detected front view index: 9 (original file: 009.png)
DINO similarity: Candidate1: 0.6015, Candidate2: 0.8942
[09ec2fa1e8152384cd7115cf3504ba1dde8e7e75f301cccf97fc8b91ea

## Equal Scaling
- Adapt both images to equal scaling 
- Width of original image is set and generated images width is set to the same value while keeping the aspect ration fixed
- Objects are maximized in the image

### Equal Scaling for multiple folders
- Folder Structure is:
      ground_truth_parent/
          <object_id_1>/
              000.png, 001.png, ..., 011.png
          <object_id_2>/
              ...
      generated_parent/
          <object_id_1>/
              000.png, 001.png, ..., 011.png
          <object_id_2>/
              ...

In [7]:
from preprocessing.image_processing import process_equal_scaling, process_equal_scaling_structure

process_equal_scaling_structure(
    ground_truth_parent="example_data/Meshfleet_Eval/Ground_Truth",
    generated_parent="example_data/Meshfleet_Eval/Results_002",
    canvas_size=(768,768),
    fill_ratio=0.9
)

Processed 0075272ea45f9d49edd2caaead93b29cac046ec537ef8c59344ca5ec4c827a65/000.png: gt_w=238, gen_w=324, scale_factor_gen=0.735, scale_factor_final=2.904
Processed 0075272ea45f9d49edd2caaead93b29cac046ec537ef8c59344ca5ec4c827a65/001.png: gt_w=341, gen_w=464, scale_factor_gen=0.735, scale_factor_final=2.027
Processed 0075272ea45f9d49edd2caaead93b29cac046ec537ef8c59344ca5ec4c827a65/002.png: gt_w=424, gen_w=588, scale_factor_gen=0.721, scale_factor_final=1.630
Processed 0075272ea45f9d49edd2caaead93b29cac046ec537ef8c59344ca5ec4c827a65/003.png: gt_w=449, gen_w=622, scale_factor_gen=0.722, scale_factor_final=1.539
Processed 0075272ea45f9d49edd2caaead93b29cac046ec537ef8c59344ca5ec4c827a65/004.png: gt_w=425, gen_w=593, scale_factor_gen=0.717, scale_factor_final=1.626
Processed 0075272ea45f9d49edd2caaead93b29cac046ec537ef8c59344ca5ec4c827a65/005.png: gt_w=364, gen_w=467, scale_factor_gen=0.779, scale_factor_final=1.899
Processed 0075272ea45f9d49edd2caaead93b29cac046ec537ef8c59344ca5ec4c827a65/0

### Equal Scaling for a folder pair with no subfolders

In [ ]:
original_folder = "example_data/examples_08082025/Car_Sample_2/original"
generated_folder = "example_data/examples_08082025/Car_Sample_2/generated_side"

out_original_folder = "example_data/examples_08082025/Car_Sample_2/scaled/original"
out_generated_folder = "example_data/examples_08082025/Car_Sample_2/scaled/generated_side"

process_equal_scaling(original_folder, generated_folder,
                        out_original_folder, out_generated_folder,
                        canvas_size=(768,768), fill_ratio=0.95)


Processed diagonal_front.png: orig_w=727, gen_w=585, scale_factor_gen=1.243, scale_factor_final=1.004
Processed front_view.png: orig_w=470, gen_w=379, scale_factor_gen=1.240, scale_factor_final=1.552
Processed rear_view.png: orig_w=447, gen_w=366, scale_factor_gen=1.221, scale_factor_final=1.632
Processed sideview.png: orig_w=926, gen_w=797, scale_factor_gen=1.162, scale_factor_final=0.788


## Concatenate images into grid
- Just for visualization

In [43]:
from preprocessing.image_processing import concatenate_images

output_file_name = "concatenated1.png"
concatenate_images(
    "/home/q613871/Projects/Trellis_Playground/TRELLIS/assets/Meshfleet_Eval/Results_006", 
    output_image=output_file_name, 
    rows = 2, 
    target_identifier = "000.png",
    crop_width_percent=30,
    crop_height_percent=50)

Concatenated image saved as 'concatenated1.png'


## Manual Image Reordering
- Can be used to manually restructure a subfolder --> Given index will be set as the new 000.png

In [ ]:
from preprocessing.image_processing import reorder_images_in_folder

target_folder = "assets/Meshfleet_Eval/019d5adeba1cc38c1dbc550b0306d008272765d13c49dec2e08a4d1276c1cca1"
reorder_images_in_folder(target_folder, 6)

## Viewpoint Alignment
- Due to the front view detection it can happen that the viewpoints are swapped which leads to the vehicle being "rotated" in the wrong direction
- With this function the user can swap the direction of the rotation by swapping the images

In [10]:
from preprocessing.image_processing import restructure_images
restructure_images("example_data/Meshfleet_Eval/Results_002")

Processing folder: example_data/Meshfleet_Eval/Results_002/043c94a4e37fc017a9c602bb6a28e8f404b1e99e8c4f3c9a6d4e0e997a8f9359
Swapped 001.png and 011.png in example_data/Meshfleet_Eval/Results_002/043c94a4e37fc017a9c602bb6a28e8f404b1e99e8c4f3c9a6d4e0e997a8f9359
Swapped 002.png and 010.png in example_data/Meshfleet_Eval/Results_002/043c94a4e37fc017a9c602bb6a28e8f404b1e99e8c4f3c9a6d4e0e997a8f9359
Swapped 003.png and 009.png in example_data/Meshfleet_Eval/Results_002/043c94a4e37fc017a9c602bb6a28e8f404b1e99e8c4f3c9a6d4e0e997a8f9359
Swapped 004.png and 008.png in example_data/Meshfleet_Eval/Results_002/043c94a4e37fc017a9c602bb6a28e8f404b1e99e8c4f3c9a6d4e0e997a8f9359
Swapped 005.png and 007.png in example_data/Meshfleet_Eval/Results_002/043c94a4e37fc017a9c602bb6a28e8f404b1e99e8c4f3c9a6d4e0e997a8f9359
Processing folder: example_data/Meshfleet_Eval/Results_002/011880bbcfe10922fac6888eddacacfa1daec200178c977b3d6beb0e6c8802c6
Swapped 001.png and 011.png in example_data/Meshfleet_Eval/Results_002/0